# MNIST classification with SVM
We will try to use SVM to classify the numbers in the MNIST dataset:
>This exercise is inspired in: https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html

In [2]:
# Importing the needed libraries
import os
import struct
import numpy as np
import matplotlib.pyplot as plt

# tells matplotlib to embed plots within the notebook
%matplotlib inline

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

## Load the dataset

In [3]:
# This function will open whatever labels mnist file and return the data
def load_mnist_labels(path):
    # Open the file rb means read only + binary
    with open(path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)
    return labels

# This function will open whatever images mnist file and return the data
# 
def load_mnist_images(path):
    # Open the file rb means read only + binary
    with open(path, 'rb') as imgpath:
        magic, n, rows, cols = struct.unpack('>IIII', imgpath.read(16))
        images_data = np.fromfile(imgpath, dtype=np.uint8)
        # with reshape with reorganize the array into n rows, 
        # where every row is the image data flat 
        images = np.reshape(images_data, (n, rows*cols))
    return images

In [4]:
mnistFolder=r"..\ex3 - MultiClass Classification + NN intro\MNIST database"
trainImages = mnistFolder + '\\train-images.idx3-ubyte'
trainLabels = mnistFolder + '\\train-labels.idx1-ubyte'
testImages = mnistFolder + '\\t10k-images.idx3-ubyte'
testLabels = mnistFolder + '\\t10k-labels.idx1-ubyte'

# labels = load_mnist_labels(testLabels)
X_train = load_mnist_images(trainImages)
y_train = load_mnist_labels(trainLabels)
X_test = load_mnist_images(testImages)
y_test = load_mnist_labels(testLabels)

## Train and test the model

In [5]:
svm = SVC(gamma=0.001)
svm.fit(X_train,y_train)

SVC(gamma=0.001)

In [7]:
pred = svm.predict(X_test)
print('Test accuracy: %.3f' %svm.score(X_test, y_test))

Test accuracy: 0.114


## Train the model with GridSearchCV

In [ ]:
param_range = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
param_grid = [{'C' : param_range,
               'kernel': ['linear']},
              {'C' : param_range,
               'gamma': param_range,
               'kernel': ['rbf']}]

 
gs = GridSearchCV(estimator=SVC(random_state=1),
                 param_grid=param_grid,
                 scoring='accuracy',
                 cv=10,
                 refit=True,
                 n_jobs=-1,
                 verbose=3)

gs.fit(X_train, y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


In [ ]:
gs.best_params_, gs.best_score_

### Predict the Test Dataset

In [ ]:
pred = gs.predict(Xtest)
print('Test accuracy: %.3f' %gs.score(Xtest, ytest))

## Check the cases not predicted properly

In [ ]:
Check_cases = 25

# First we identify the cases where the prediction 
# is not correct - misclassified 
misclX = Xtest[pred != ytest]
correct_label = ytest[pred != ytest]
miscl_label = pred[pred !=ytest]

miscl_m = miscl_label.size

# We get a testCases random datapoints. First we get a number of testCases indices out of m
rand_indices = np.random.choice(miscl_m, Check_cases, replace=False)

sel_misclX = misclX[rand_indices, :]
sel_correct_label = correct_label[rand_indices]
sel_miscl_label = miscl_label[rand_indices]

In [ ]:
fig, ax = plt.subplots(nrows = 5, ncols = 5, sharex = True, sharey = True)
ax = ax.flatten()
for i in range(Check_cases):
    img = sel_misclX[i].reshape(28,28)
    ax[i].imshow(img, cmap = 'Greys', interpolation = 'nearest')
    ax[i].set_title('%d) t:%d p:%d' % (i+1,sel_correct_label[i], sel_miscl_label[i]))
ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()